# This is a section to be ran in main program
## you cannot run this on its own, needs more context

### lock class

In [1]:
env = simpy.Environment()
FG = nx.read_shp(os.path.join(location_graph, name_graph), 
                 simplify=True)
H_G = nx.Graph()
lat_lon_to_index = {}
edge_id_counter = 0

HEIGHT_RESTRICED_NODES = {'2': 8.7}
# first node is closed, rest is locks which we want to close on one side for now
CLOSED_NODES = [143, 365, 73, 208]

NameError: name 'simpy' is not defined

In [ ]:
class Lock():
    def __init__(self, args):
        self.nodes = args['nodes']
        self.operation_time = 0
        self.length = args['length']
        self.width = args['width']
        self.doors_open = args['doors_open']
        self.doors_close = args['doors_close']
        self.operating_time = args['operating_time']
        self.resource = args['resource']


## Create lock entries in the existing graph

In [ ]:
LOCKS = [
    Lock({'nodes':[178,179],'length':280,'width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':1440, 'resource':simpy.PriorityResource(env, capacity = 1)}), 
    Lock({'nodes':[220,221], 'length':280,'width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':720, 'resource':simpy.PriorityResource(env, capacity = 1)}), 
    Lock({'nodes':[155,156], 'length':320,'width':24, 'doors_open': 60, 'doors_close':60, 'operating_time':840, 'resource':simpy.PriorityResource(env, capacity = 1)})
]

In [ ]:

# calculate distance between two points
def calculate_distance(orig, dest):
    wgs84 = pyproj.Geod(ellps='WGS84')
    distance = wgs84.inv(shapely.geometry.asShape(orig).x, shapely.geometry.asShape(orig).y, 
                         shapely.geometry.asShape(dest).x, shapely.geometry.asShape(dest).y)[2]
    return distance


def get_lock(node_one, node_two):
    for lock in LOCKS:
        nodes_lock = lock.nodes
        if node_one in nodes_lock and node_two in nodes_lock:
            return lock
    return None

def get_height_restriction(index):
    '''
    Restriction in meters for a node, vessel can only pass if height < restriction
    '''
    if (str(index) in HEIGHT_RESTRICED_NODES.keys()):
        print("found restriction {}".format(HEIGHT_RESTRICED_NODES[str(index)]))
        return HEIGHT_RESTRICED_NODES[str(index)]
    return 99

def get_closed(index):
    if index in CLOSED_NODES:
        return True
    return False

          
def do_nodes():
    for i, node in enumerate(FG.nodes(data = True)):
        H_G.add_node(i, pos = node[0], 
                        height_restriction = get_height_restriction(i),
                        closed = get_closed(i),
                        name = "Node {}".format(i),
                        geometry = shapely.geometry.Point(node[0][0], node[0][1]))
        lat_lon_to_index[node[0]] = i

def do_edges():
    for edge in FG.edges(data=True):
        edge_type = 'Canal'
        lock = get_lock(lat_lon_to_index[edge[0]],lat_lon_to_index[edge[1]])
        edge_attribute = None
        if (lock != None):
            edge_type = 'Lock'
            edge_attribute = lock
            print(lock)
        H_G.add_edge(lat_lon_to_index[edge[0]], 
                     lat_lon_to_index[edge[1]],
                     Object=edge_type,
                     dis= calculate_distance(nx.get_node_attributes(H_G, 'geometry')[lat_lon_to_index[edge[1]]], 
                                             nx.get_node_attributes(H_G, 'geometry')[lat_lon_to_index[edge[0]]]),
                     weight = 1,
                     attribute = edge_attribute
                    )
        if (edge_type == 'Lock'):
            H_G.edges[lat_lon_to_index[edge[0]],lat_lon_to_index[edge[1]]]["Resources"] = lock.resource
        
do_nodes()
do_edges()

                     
FG = H_G.to_directed()

In [ ]:
def get_lock_edges():
    for edge in H_G.edges(data=True):
        if edge[2]['Object'] == 'Lock':
            print(edge)
            print("Has node connecting to..")
            node_1 = edge[0]
            node_1 = edge[1]
            for edge_connect in H_G.edges(data=True):
                if edge_connect[0] == edge[0] or edge_connect[0] == edge[1] or edge_connect[1] == edge[1] or edge_connect[1] == edge[0]:
                    print(edge_connect)
                    print(H_G.nodes(data=True)[edge[0]])
                    print(H_G.nodes(data=True)[edge[1]])

In [ ]:
"""
for edge in FG.edges(data=True):
    if edge[0] == 178 and edge[1] == 179:
        print(edge)
    if edge[1] == 178 and edge[0] == 179:
        print(edge)
    if edge[0] == 317 and edge[1] == 365:
        print(edge)
    if edge[1] == 365 and edge[0] == 317:
        print(edge)
"""